In [1]:
from visualizer import get_local
get_local.activate()

import os
ROOT_PATH="/data/aofei"
os.environ["CUDA_VISIBLE_DEVICES"] = '3'
os.environ["TRANSFORMERS_CACHE"] = f"{ROOT_PATH}/huggingface_cache/transformers"
os.environ["HF_HOME"] = f"{ROOT_PATH}/huggingface_cache/transformers"

from PIL import Image
import seaborn as sns
import torch
import torch.nn.functional as F
import json
import copy
import numpy as np

import sys
sys.path.append("/home/avc6555/research/MedH/Mitigation/LVLMs/llava-med")

from llava import LlavaLlamaForCausalLM
from llava.conversation import conv_templates
from llava.utils import disable_torch_init
from transformers import CLIPVisionModel, CLIPImageProcessor, StoppingCriteria
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig

from PIL import Image
import random
import math

from visualize.utils import show_image_relevance, compute_ca_loss

/data/aofei/conda/env/medh/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/data/aofei/conda/env/medh/lib/python3.10/site-packages/transformers/utils/hub.py:123: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
/data/aofei/conda/env/medh/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/data/aofei/conda/env/medh/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytr

In [2]:
model = None
torch.cuda.empty_cache()

In [3]:
# load model
ROOT_PATH="/data/aofei"
model_name = f"{ROOT_PATH}/LLM/llava_med"

DEFAULT_IMAGE_TOKEN = "<image>"
DEFAULT_IMAGE_PATCH_TOKEN = "<im_patch>"
DEFAULT_IM_START_TOKEN = "<im_start>"
DEFAULT_IM_END_TOKEN = "<im_end>"

model_name = os.path.expanduser(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = LlavaLlamaForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, use_cache=True).cuda()
# print("vision tower", model.config.mm_vision_tower)
image_processor = CLIPImageProcessor.from_pretrained(model.config.mm_vision_tower, torch_dtype=torch.float16)
vision_tower = model.model.vision_tower[0]
vision_tower.to(device='cuda', dtype=torch.float16)

mm_use_im_start_end = getattr(model.config, "mm_use_im_start_end", False)
tokenizer.add_tokens([DEFAULT_IMAGE_PATCH_TOKEN], special_tokens=True)
if mm_use_im_start_end:
    tokenizer.add_tokens([DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN], special_tokens=True)

vision_config = vision_tower.config
vision_config.im_patch_token = tokenizer.convert_tokens_to_ids([DEFAULT_IMAGE_PATCH_TOKEN])[0]
vision_config.use_im_start_end = mm_use_im_start_end
if mm_use_im_start_end:
    vision_config.im_start_token, vision_config.im_end_token = tokenizer.convert_tokens_to_ids([DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN])
image_token_len = (vision_config.image_size // vision_config.patch_size) ** 2
print(image_token_len, "token length")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/data/aofei/conda/env/medh/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

256 token length


In [10]:
import clip
get_local.clear()
torch.cuda.empty_cache()

H, W = 16, 16
_, preprocess = clip.load("ViT-B/32", device='cpu', jit=False)

class KeywordsStoppingCriteria(StoppingCriteria):
    def __init__(self, keywords, tokenizer, input_ids):
        self.keywords = keywords
        self.tokenizer = tokenizer
        self.start_len = None
        self.input_ids = input_ids
        

    def __call__(self, output_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        if self.start_len is None:
            self.start_len = self.input_ids.shape[1]
        else:
            outputs = self.tokenizer.batch_decode(output_ids[:, self.start_len:], skip_special_tokens=True)[0]
            for keyword in self.keywords:
                if keyword in outputs:
                    return True
        return False

In [4]:
#LOAD LORA
from llava.model.moe_llava import LoRA_MOE_FFN, LoRA_MOE_QK, LoRA_MOE_QK_old

query_expert_num, key_expert_num, rank, alpha = 4, 8, 16, 8
# query_expert_num, key_expert_num, rank, alpha = 8, 16, 16, 8
dense_q_moe = True

peft_path = "/data/aofei/hallucination/mitigation/Slake/llava_med/gt_exp/only_gt/all_expert_4_8_rank16/lora_bbox_0.2/epoch9_seed4/checkpoints"

if peft_path is not None and len(str(peft_path))>4:
    from peft import PeftModel
    print(f"Loading LoRA weights from {peft_path}")
    model = PeftModel.from_pretrained(model, peft_path)
    print(f"Merging weights")
    model = model.merge_and_unload()
    
    moe_path = os.path.join(peft_path, "non_lora_trainables.bin")
    moe_state_dict = torch.load(moe_path, map_location='cuda')
    if len(moe_state_dict.keys()) > 32:
        print("load MoE parameters!")
        num_layers = len(model.base_model.layers)
        for i in range(num_layers):
            original_q = model.base_model.layers[i].self_attn.q_proj
            model.base_model.layers[i].self_attn.q_proj = \
                LoRA_MOE_QK(args=None,
                    lora_rank=rank,
                    lora_alpha=alpha,
                    num_experts=query_expert_num,
                    original_module=original_q,
                    dense_moe=dense_q_moe)
        for i in range(num_layers):
            original_k = model.base_model.layers[i].self_attn.k_proj
            model.base_model.layers[i].self_attn.k_proj = \
                LoRA_MOE_QK_old(args=None,
                    lora_rank=rank,
                    lora_alpha=alpha,
                    num_experts=key_expert_num,
                    top_moe_experts=2,
                    original_module=original_k)
        new_state_dict = {}
        for key, value in moe_state_dict.items():
            # Replace "base_model.model" with an empty string to remove it
            new_key = key.replace("base_model.model", "")
            if new_key.startswith("."):
                new_key = new_key[1:]
            new_state_dict[new_key] = value.to("cuda")
            # new_state_dict[new_key] = value
        model.load_state_dict(new_state_dict, strict=False)
        model = model.to("cuda")
        for key in new_state_dict.keys():
            assert torch.equal(model.state_dict()[key], new_state_dict[key]), f"Mismatch in {key}"
        print("Subset loaded successfully.")
        print('Convert to FP16...')
        model.to(torch.float16)

Loading LoRA weights from /data/aofei/hallucination/mitigation/Slake/llava_med/gt_exp/only_gt/all_expert_4_8_rank16/lora_bbox_0.2/epoch9_seed4/checkpoints
Merging weights
load MoE parameters!
Subset loaded successfully.
Convert to FP16...


### Test interpretability

In [5]:
def measure_interpret(model, input_ids, images, mask, threshold=0.2):
    # Extract the image token index
    img_token_idx = int(torch.where(input_ids.squeeze(0) == 32001)[0][0])

    # Generate model outputs with attentions
    outputs = model.generate(
        input_ids=input_ids,
        images=images,
        do_sample=False,
        temperature=0.7,
        max_new_tokens=20,
        output_attentions=True,
        output_scores=True,
        return_dict_in_generate=True
    )

    # Retrieve cached attention maps
    cache = get_local.cache
    ori_attention_maps = [i for i in cache['LlamaSdpaAttention.forward'] if i is not None]
    attention_maps = [att for i, att in enumerate(ori_attention_maps) if att.shape[-2] > 1]

    # Compute mean attention map
    mean_att = torch.cat(attention_maps, 0).mean(0)
    target2img_rel = mean_att[:, img_token_idx+mask.shape[0]*mask.shape[1]:, img_token_idx:img_token_idx+mask.shape[0]*mask.shape[1]].mean(axis=0).mean(axis=0).unsqueeze(0)

    # Reshape the attention map to match the image dimensions
    avg_attention_map = target2img_rel.reshape(mask.shape)

    # Normalize the attention map to [0, 1]
    avg_attention_map = (avg_attention_map - avg_attention_map.min()) / (avg_attention_map.max() - avg_attention_map.min())

    # Define a function to compute Coverage Score
    def compute_coverage(mask, attention_map, threshold=threshold):
        thresholded_attention = (attention_map >= threshold).float()
        coverage = (mask * thresholded_attention).sum() / mask.sum()
        return coverage.item()

    # Define a function to compute Intensity Alignment
    def compute_intensity_alignment(mask, attention_map):
        intensity_alignment = (mask * attention_map).sum() / mask.sum()
        return intensity_alignment.item()

    # Compute the two metrics
    coverage_score = compute_coverage(mask, avg_attention_map)
    intensity_alignment = compute_intensity_alignment(mask, avg_attention_map)

    # Clear cache and return output
    get_local.clear()
    torch.cuda.empty_cache()

    return {
        "output": tokenizer.decode(outputs["sequences"][0]).strip(),
        "coverage_score": coverage_score,
        "intensity_alignment": intensity_alignment,
        "mask":mask.detach().cpu(),
        "attention_map": avg_attention_map.detach().cpu()
    }


In [6]:
import json

test_gt_path = "/data/aofei/hallucination/Slake/data/with_gt/test_with_gt.json"
with open(test_gt_path) as f:
    test_gt = json.load(f)
test_gt[0], len(test_gt)

({'image': 'xmlab102/source.jpg',
  'id': 11941,
  'organ': 'Lung',
  'answer_type': 'OPEN',
  'conversations': [{'from': 'human',
    'value': '<image>\nWhat diseases are included in the picture?'},
   {'from': 'gpt', 'value': 'Lung Cancer'}],
  'bboxes': [[144, 150, 9, 11]],
  'gt_bboxes': [[144, 150, 9, 11]]},
 206)

In [7]:
from tqdm import tqdm

In [8]:
get_local.clear()
torch.cuda.empty_cache()
inference_results = []

In [11]:
import torchvision.transforms as transforms
n_px = 224
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(n_px, interpolation=transforms.InterpolationMode.NEAREST),
    transforms.CenterCrop(n_px),
    transforms.Resize(H, interpolation=transforms.InterpolationMode.NEAREST),
    transforms.ToTensor(),
    ])

In [14]:

ih, iw = 256, 256
for idx, item in tqdm(list(enumerate(test_gt[5:]))):
    get_local.clear()
    torch.cuda.empty_cache()
    _id = item['id']
    question = item['conversations'][0]['value']
    image_path = os.path.join("/data/aofei/hallucination/Slake/imgs",item['image']) 
    gt_answer = item['conversations'][1]['value']
    inference_item = {"question_id":_id, 'question': question, 'gt': gt_answer}
    gt_bboxes = item['gt_bboxes']

    qs = question.replace(DEFAULT_IMAGE_TOKEN, '').strip()
    image = Image.open(image_path)
    image_tensor = image_processor.preprocess(image, return_tensors='pt')['pixel_values'][0]
    images = image_tensor.unsqueeze(0).half().cuda()
    if getattr(model.config, 'mm_use_im_start_end', False):
        qs = DEFAULT_IM_START_TOKEN + DEFAULT_IMAGE_PATCH_TOKEN * image_token_len + DEFAULT_IM_END_TOKEN + '\n' +  qs
    else:
        qs = DEFAULT_IMAGE_PATCH_TOKEN * image_token_len + '\n' + qs
    cur_prompt = '<image>' + '\n' +  qs
    conv_mode = "simple"
    conv = conv_templates[conv_mode].copy()
    conv.messages = []
    conv.append_message(conv.roles[0], qs)
    prompt = conv.get_prompt()
    inputs = tokenizer([prompt])
    input_ids = torch.as_tensor(inputs.input_ids).cuda()
    mask = torch.zeros(size=(ih, iw))
    for bbox in gt_bboxes:
        x_min, y_min, x_max, y_max = int(bbox[0]), int(bbox[1]), int(bbox[0]) + int(bbox[2]), int(bbox[1]) + int(bbox[3])
        mask[y_min: y_max, x_min: x_max] = 1
    mask = transform(mask.numpy())[0]
    mask = mask.cuda() if torch.cuda.is_available() else mask
    output = measure_interpret(model, input_ids, images, mask, threshold=0.05)
    
    inference_results.append(output)
    get_local.clear()
    torch.cuda.empty_cache()

100%|██████████| 201/201 [04:52<00:00,  1.46s/it]


In [15]:
len(inference_results)

206

In [16]:
# inference_results
def compute_coverage(mask, attention_map, threshold=0.15):
    thresholded_attention = (attention_map >= threshold).float()
    coverage = (mask * thresholded_attention).sum() / mask.sum()
    return coverage.item()

# Define a function to compute Intensity Alignment
def compute_intensity_alignment(mask, attention_map):
    intensity_alignment = (mask * attention_map).sum() / mask.sum()
    return intensity_alignment.item()

new_coverage_scores = []
new_intensity_alignment = []
for i in inference_results:
    mask = i['mask']
    attention_map = i['attention_map']
    coverage_score = compute_coverage(mask, attention_map)
    intensity_alignment = compute_intensity_alignment(mask, attention_map)
    new_coverage_scores.append(coverage_score)
    new_intensity_alignment.append(intensity_alignment)

np.nanmean(new_coverage_scores), np.nanmean(new_intensity_alignment)

(0.22666302749222547, 0.11998396067313813)

In [17]:
# inference_results

import numpy as np

average_coverage = np.nanmean([i['coverage_score'] for i in inference_results])
average_intensity = np.nanmean([i['intensity_alignment'] for i in inference_results])
average_coverage, average_intensity

(0.5922559349980187, 0.12084772437810898)

In [25]:
for i in inference_results:
    i['text'] = i['output']

with open("gt_results/ori_inference.jsonl", "w") as f:
    for item in inference_results:
        f.write(json.dumps(item)+"\n")

In [26]:
import subprocess

# Define the root path
ROOT_PATH="/data/aofei"
dataset="Slake"

# Construct the command
command = [
    "python", "/home/avc6555/research/MedH/Mitigation/LVLMs/llava-med/llava/eval/run_eval.py",
    "--gt", f"{ROOT_PATH}/hallucination/{dataset}/data/with_gt/test_with_gt.json",
    "--pred", f"/home/avc6555/research/MedH/visualize/llava_visualize/case_llava_med/gt_results/ori_inference.jsonl",
    "--eval_res", f"/home/avc6555/research/MedH/visualize/llava_visualize/case_llava_med/gt_results/eval_res_ori.txt"
]

# Run the command
result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# Print the output
# print("Output:")
# print(result)
